### Language Model and Application for Spelling Error Correction


In [ ]:
import string, re, math
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict, Counter
import numpy as np
from difflib import get_close_matches
import joblib

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Cải thiện mô hình sử dụng làm mượt nội suy (Interpolation smoothing) với phương pháp "Stupid Backoff"
- Sử dụng mô hình NGram
- Làm mượt xác suất bằng phương pháp "Stupid Backoff"

In [ ]:
# Tải dữ liệu
with open("tedtalk.txt", "r", encoding="utf-8") as f:
    docs = f.read()
# Tiền xử lý
def preprocess(sentence):
    # Giữ lại chữ cái, chữ số, và dấu nháy đơn
    text = re.sub(r"[^A-Za-z0-9'\s]", " ", sentence)
    # Chuyển về chữ thường
    text = text.lower()
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r"\s+", " ", text).strip()
    return word_tokenize(text)
# Tách câu
sentences = sent_tokenize(docs)
# Tiền xử lý từng câu
tokenized = [preprocess(s) for s in sentences]
print("Số câu:", len(tokenized))

Số câu: 444404


In [ ]:
class NgramModel:
    def __init__(self, n):
        self.n = n # Bậc của mô hình
        self.model = defaultdict(Counter)  # Lưu trữ n-gram counts
        self.vocab = set()  # Tập từ vựng của mô hình

    def train(self, sentences):
        # sentences: list các câu, mỗi câu là list token
        for tokens in sentences:
            self.vocab.update(tokens)   # cập nhật vocab
            padded = ["<s>"]*(self.n-1) + tokens + ["</s>"]  # padding đầu/cuối
            for i in range(len(padded)-self.n+1):
                context = tuple(padded[i:i+self.n-1])  # ngữ cảnh
                word = padded[i+self.n-1]  # từ cần dự đoán
                self.model[context][word] += 1

    # Tính xác suất có điều kiện với Laplace smoothing
    def prob(self, context, word):
        context_counts = self.model[context]
        V = len(self.vocab)
        return (context_counts[word] + 1) / (sum(context_counts.values()) + V)
    # Tính xác suất của một câu
    def sentence_prob(self, sentence):
        tokens = ["<s>"]*(self.n-1) + preprocess(sentence) + ["</s>"]
        prob = 1.0
        for i in range(len(tokens)-self.n+1):
            context = tuple(tokens[i:i+self.n-1])
            word = tokens[i+self.n-1]
            prob *= self.prob(context, word)
        return prob

In [ ]:
unigram = NgramModel(1) # Mô hình 1-gram
bigram = NgramModel(2) # Mô hình 2-gram
trigram = NgramModel(3) # Mô hình 3-gram
# Huấn luyện mô hình
unigram.train(tokenized)
bigram.train(tokenized)
trigram.train(tokenized)
print("Mô hình Laplace đã được huấn luyện!")

Mô hình Laplace đã được huấn luyện!


In [ ]:
# Tính perplexity
def perplexity(model, sentence):
    tokens = ["<s>"]*(model.n-1) + preprocess(sentence) + ["</s>"] # padding
    log_prob = 0
    N = len(tokens) - (model.n-1) # số từ thực tế
    for i in range(len(tokens)-model.n+1): # duyệt qua từng n-gram
        context = tuple(tokens[i:i+model.n-1])
        word = tokens[i+model.n-1] # từ cần dự đoán
        p = model.prob(context, word) # xác suất có điều kiện
        log_prob += math.log(p)
    return math.exp(-log_prob/N)

sentence = "the climate crisis is real"
print("1-gram perplexity:", perplexity(unigram, sentence))
print("2-gram perplexity:", perplexity(bigram, sentence))
print("3-gram perplexity:", perplexity(trigram, sentence))

1-gram perplexity: 401.6450114268086
2-gram perplexity: 564.1016367106075
3-gram perplexity: 2408.989014775055


In [ ]:
# Mô hình Stupid Backoff
class StupidBackoffModel(NgramModel):
    def __init__(self, n, alpha=0.4):
        super().__init__(n) # Gọi hàm khởi tạo của lớp cha
        self.alpha = alpha # Hệ số giảm bậc
        self.lower_order = None # Mô hình bậc thấp hơn

    def link_lower_order(self, lower_model):
        self.lower_order = lower_model

    def prob(self, context, word):
        if self.model[context][word] > 0: # Nếu n-gram có trong mô hình
            return self.model[context][word] / sum(self.model[context].values())
        elif self.lower_order:
            return self.alpha * self.lower_order.prob(context[1:], word)
        else:
            return 1 / len(self.vocab)
# Huấn luyện mô hình Stupid Backoff
sb_trigram = StupidBackoffModel(3)
sb_bigram = StupidBackoffModel(2)
sb_unigram = StupidBackoffModel(1)

sb_trigram.train(tokenized)
sb_bigram.train(tokenized)
sb_unigram.train(tokenized)
# Liên kết các mô hình với nhau
sb_trigram.link_lower_order(sb_bigram)
sb_bigram.link_lower_order(sb_unigram)
print("Mô hình Stupid Backoff đã được huấn luyện!")

Mô hình Stupid Backoff đã được huấn luyện!


### So sánh giữa 2 phương pháp làm mượt Laplace vs Stupid Backoff

In [ ]:

correct = "the climate crisis is real"
incorrect = "climate the crisis real is"

print("=== Laplace Bigram ===")
print("Correct:", bigram.sentence_prob(correct))
print("Incorrect:", bigram.sentence_prob(incorrect))

print("\n=== Stupid Backoff Bigram ===")
print("Correct:", sb_bigram.sentence_prob(correct))
print("Incorrect:", sb_bigram.sentence_prob(incorrect))

print("\n=== Laplace Trigram ===")
print("Correct:", trigram.sentence_prob(correct))
print("Incorrect:", trigram.sentence_prob(incorrect))

print("\n=== Stupid Backoff Trigram ===")
print("Correct:", sb_trigram.sentence_prob(correct))
print("Incorrect:", sb_trigram.sentence_prob(incorrect))


=== Laplace Bigram ===
Correct: 3.10352923774124e-17
Incorrect: 1.7791565158621212e-23

=== Stupid Backoff Bigram ===
Correct: 6.311251761700601e-12
Incorrect: 3.598266099224033e-19

=== Laplace Trigram ===
Correct: 5.1167132917041455e-21
Incorrect: 5.153658419123097e-29

=== Stupid Backoff Trigram ===
Correct: 1.1463775565573334e-10
Incorrect: 3.684624485605411e-21


- Từ phần kết quả ta thấy được:
+ Câu đúng có xác suất cao hơn câu sai ở cả hai phương pháp.
+ Stupid Backoff thường phân biệt rõ hơn giữa đúng và sai.

### Dự đoán từ tiếp theo dựa vào ngữ cảnh cho trước

In [ ]:
def predict_next_word(model, context, topk=5): # Dự đoán từ dựa trên ngữ cảnh
    context = tuple(context[-(model.n-1):]) # lấy (n-1) từ cuối của ngữ cảnh
    probs = {w: model.prob(context, w) for w in model.vocab}
    return sorted(probs.items(), key=lambda x: x[1], reverse=True)[:topk]

print(predict_next_word(sb_trigram, ["the", "climate"]))

[('crisis', 0.2831050228310502), ('change', 0.091324200913242), ('is', 0.0410958904109589), ('system', 0.0273972602739726), ('of', 0.0182648401826484)]


### Sử dụng hàm tính khoảng cách để sửa lại từ bị sai với thư viện difflib (from difflib import get_close_matches)

In [ ]:
def correct_word(word, vocab, n=1):
    matches = get_close_matches(word, vocab, n=n) # Tìm từ gần đúng nhất
    return matches[0] if matches else word

print(correct_word("devolopmentt", list(sb_trigram.vocab)))

development


In [ ]:
joblib.dump(unigram, "laplace_unigram.pkl")
joblib.dump(bigram, "laplace_bigram.pkl")
joblib.dump(trigram, "laplace_trigram.pkl")

# Lưu mô hình Stupid Backoff
joblib.dump(sb_unigram, "sb_unigram.pkl")
joblib.dump(sb_bigram, "sb_bigram.pkl")
joblib.dump(sb_trigram, "sb_trigram.pkl")